In [3]:
import sqlite3
#import pymongo
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta, MO
from matplotlib import colors
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output


In [4]:
#doc_db_clinic = conn_doc_db["doc_db_clinic"]
#print(doc_db_clinic.list_collection_names())
#doc_db_clinic_med_records = doc_db_clinic["med_records"]

#Importing the library to connect to the database. 
import sqlite3 
#Creating a connection to the database. 
conn = sqlite3.connect('ehr.db') 
#A cursor holds resources for a query. Several cursors can be created for a connection. 
cur = conn.cursor()


In [5]:
cur.execute("PRAGMA foreign_keys = ON")

In [6]:
createTblPatient = "CREATE TABLE PATIENT ( " + \
"PATIENT_ID varchar(100) NOT NULL PRIMARY KEY, " + \
"FIRST_NAME varchar(255) NOT NULL, " + \
"LAST_NAME varchar(255) NOT NULL, " + \
"AGE integer NOT NULL, " + \
"GENDER varchar(100) NOT NULL, " + \
"POST_CODE varchar(100) NOT NULL, " + \
"MARTIAL_STATUS varchar(100) NOT NULL, " + \
"ANNUAL_INCOME integer NOT NULL, " + \
"BIRTH_YEAR integer NOT NULL, " + \
"PHONE_NUMBER integer NOT NULL); "

cur.execute(createTblPatient)

In [7]:
createTblEncounter = "CREATE TABLE ENCOUNTER ( " + \
"ENCOUNTER_ID varchar(100) NOT NULL PRIMARY KEY , " + \
"PATIENT_ID varchar(100) NOT NULL , " + \
"DOCTOR_ID varchar(100) NOT NULL, " + \
"ADMISSION_STATUS varchar(200) NOT NULL, " + \
"BP_SYSTOLIC integer NOT NULL, " + \
"BP_DIASTOLIC integer NOT NULL, " + \
"PULSE integer NOT NULL, " + \
"HEIGHT integer NOT NULL, " + \
"WEIGHT integer NOT NULL, " + \
"BMI integer NOT NULL, FOREIGN KEY(PATIENT_ID) references PATIENT(PATIENT_ID)on Update CASCADE on Delete Set Null, FOREIGN KEY(DOCTOR_ID) references HCP(DOCTOR_ID)on Update CASCADE on Delete Set Null); "

cur.execute(createTblEncounter)


In [8]:
createTblDiagnosis = "CREATE TABLE DIAGNOSIS ( " + \
"DIAGNOSIS_ID varchar(100) NOT NULL PRIMARY KEY, " + \
"PATIENT_ID varchar(100) NOT NULL, " + \
"ENCOUNTER_ID varchar(100) NOT NULL, " + \
"DIAGNOSIS_CODE varchar(100) NOT NULL, " + \
"DIAGNOSIS_DESCRIPTION varchar(2000) NOT NULL, " + \
"DIAGNOSIS_DATE date NOT NULL, " + \
"DIAGNOSIS_CODE_TYPE varchar(100) NOT NULL, FOREIGN KEY(PATIENT_ID) references PATIENT(PATIENT_ID)on Update CASCADE on Delete Set Null, FOREIGN KEY(ENCOUNTER_ID) references ENCOUNTER(ENCOUNTER_ID)on Update CASCADE on Delete Set Null); "

cur.execute(createTblDiagnosis)

In [9]:
createTblProcedure = "CREATE TABLE PROCEDURE ( " + \
"PROCEDURE_ID varchar(100) NOT NULL PRIMARY KEY, " + \
"PATIENT_ID varchar(100) NOT NULL, " + \
"ENCOUNTER_ID varchar(100) NOT NULL, " + \
"PROCEDURE_CODE varchar(100) NOT NULL, " + \
"PROCEDURE_DESCRIPTION varchar(2000) NOT NULL, " + \
"PROCEDURE_START_DATE date NOT NULL, " + \
"PROCEDURE_END_DATE date NOT NULL, " + \
"PROCEDURE_CODE_TYPE varchar(100) NOT NULL, FOREIGN KEY(PATIENT_ID) references PATIENT(PATIENT_ID)on Update CASCADE on Delete Set Null, FOREIGN KEY(ENCOUNTER_ID) references ENCOUNTER(ENCOUNTER_ID)on Update CASCADE on Delete Set Null); "

cur.execute(createTblProcedure)

In [182]:
createTblHCP = "CREATE TABLE HCP ( " + \
"DOCTOR_ID varchar(100) NOT NULL PRIMARY KEY, " + \
"DOCTOR_NAME varchar(100) NOT NULL, " + \
"DEPARTMENT varchar(200) NOT NULL " + \
"); "

cur.execute(createTblHCP)

In [191]:
createTblBills = "CREATE TABLE BILLS ( " + \
"BILL_NO varchar(100) NOT NULL PRIMARY KEY, " + \
"PATIENT_ID varchar(100) NOT NULL, " + \
"ENCOUNTER_ID varchar(100) NOT NULL, " + \
"TOTAL_COST integer NOT NULL, " + \
"NET_COST integer NOT NULL, " + \
"INSURANCE_TYPE varchar(100) NOT NULL, FOREIGN KEY(PATIENT_ID) references PATIENT(PATIENT_ID)on Update CASCADE on Delete Set Null, FOREIGN KEY(ENCOUNTER_ID) references ENCOUNTER(ENCOUNTER_ID)on Update CASCADE on Delete Set Null); "

cur.execute(createTblBills)

In [193]:
createTblPharmacy = "CREATE TABLE PHARMACY ( " + \
"PURCHASE_ID varchar(100) NOT NULL PRIMARY KEY, " + \
"PATIENT_ID varchar(100) NOT NULL, " + \
"ENCOUNTER_ID varchar(100) NOT NULL, " + \
"DRUG_CODE varchar(100) NOT NULL, " + \
"DRUG_NAME varchar(200) NOT NULL, " + \
"DOSAGE varchar(100) NOT NULL, " + \
"FREQUENCY varchar(100) NOT NULL, FOREIGN KEY(PATIENT_ID) references PATIENT(PATIENT_ID)on Update CASCADE on Delete Set Null, FOREIGN KEY(ENCOUNTER_ID) references ENCOUNTER(ENCOUNTER_ID)on Update CASCADE on Delete Set Null); "

cur.execute(createTblPharmacy)

#### Inserting values into tables

In [183]:
qry="""insert into patient (PATIENT_ID, FIRST_NAME, LAST_NAME, AGE, GENDER, POST_CODE, MARTIAL_STATUS, ANNUAL_INCOME, BIRTH_YEAR, PHONE_NUMBER)  
values 
("p01", "George", "Thomas", 26, "Male", "EX41AU", "Married", 28000, 1996, 7918276264),
("p02", "Thomass", "Shelby", 36, "Male", "Bl00AE", "Married", 60000, 1986, 7403023467),
("p03", "Arthur", "Shelby", 47, "Male", "Bl00AX", "Married", 54000, 1975, 8045312789),
("p04", "Polly", "Grey", 52, "Female", "Bl00AD", "Single", 43000, 1970, 7919274364),
("p05", "Jay", "Prichet", 72, "Male", "PL00BC", "Married", 70000, 1950, 7403023467),
("p06", "Gloria", "Sebastin", 45, "Female", "EX00BT", "Married", 43000, 1977, 80125312789),
("p07", "Cameron", "Oscar", 44, "Male", "LO41AU", "Single", 32000, 1978, 7888271209),
("p08", "Claire", "Alex", 40, "Female", "BO00MT", "Married", 60000, 1974, 6409023476),
("p09", "Marius", "Tkozc", 29, "Male", "EX43AX", "Single", 59000, 1993, 78945312789),
("p10", "Justine", "leo", 22, "Male", "Bl00AR", "Single", 23000, 2000, 7561925836),
("p11", "John", "Cena", 48, "Male", "SW00BT", "Married", 65000, 1974, 6403023742),
("p12", "Rey", "Mysterio", 60, "Male", "CA00TE", "Married", 43000, 1954, 7303312798),
("p13", "Dunphy", "Luke", 46, "Male", "BR00AX", "Married", 54000, 1976, 7445412784),
("p14", "Richard", "Dare", 53, "Male", "AA11AD", "Single", 43000, 1969, 9919274366),
("p15", "Lenin", "Archimedes", 71, "Male", "OX35BC", "Married", 70000, 1951, 8403023461),
("p16", "Ronaldo", "Trump", 49, "Male", "BI11BX", "Married", 43000, 1973, 6525312700),
("p17", "Shetty", "shilpa", 33, "Female", "LU41AU", "Single", 32000, 1989, 7778271255),
("p18", "Sunny", "Leone", 41, "Female", "BL00MX", "Married", 60000, 1981, 6459023474),
("p19", "Mia", "khalifa", 25, "Female", "EX43AT", "Single", 55000, 1997, 88945312774),
("p20", "Harry", "Styles", 24, "Male", "Bl00AR", "Single", 33000, 1998, 7211925836),
("p21", "John", "Sins", 61, "Male", "WS00BT", "Married", 64000, 1953, 6663023741),
("p22", "Letty", "Diesel", 52, "Female", "CA00TE", "Married", 43000, 1970, 7733312714),
("p23", "Samuel", "Shankar", 44, "Male", "AL00AX", "Married", 54000, 1978, 6445412777),
("p24", "Dwayne", "Johnson", 54, "Male", "NC11AD", "Single", 93000, 1968, 875474368),
("p25", "Paul", "Walker", 71, "Male", "OX35BC", "Married", 76000, 1943, 8443023463),
("p26", "Rishi", "Sunakk", 80, "Male", "BO11BT", "Married", 43000, 1934, 6525312700),
("p27", "Boriss", "Thomsan", 24, "Male", "LN41AU", "Single", 12000, 1998, 7778881253),
("p28", "Nayanthara", "david", 39, "Female", "ND00MR", "Married", 45500, 1983, 6559023471),
("p29", "Wiz", "khashim", 37, "Male", "WX43AT", "Single", 25000, 1971, 88567312774),
("p30", "Napoelean", "Rambo", 92, "Male", "BE00AR", "Single", 76000, 1930, 8999925839)
;"""


cur.execute(qry)

# Commit data to database
conn.commit()


# Prepare the query String
qry="select * from patient;"

# Execute query on SQLite
cur.execute(qry)

# Display rows
rows=cur.fetchall()

for row in rows:
    print (row)


('p01', 'George', 'Thomas', 26, 'Male', 'EX41AU', 'Married', 28000, 1996, 7918276264)
('p02', 'Thomass', 'Shelby', 36, 'Male', 'Bl00AE', 'Married', 60000, 1986, 7403023467)
('p03', 'Arthur', 'Shelby', 47, 'Male', 'Bl00AX', 'Married', 54000, 1975, 8045312789)
('p04', 'Polly', 'Grey', 52, 'Female', 'Bl00AD', 'Single', 43000, 1970, 7919274364)
('p05', 'Jay', 'Prichet', 72, 'Male', 'PL00BC', 'Married', 70000, 1950, 7403023467)
('p06', 'Gloria', 'Sebastin', 45, 'Female', 'EX00BT', 'Married', 43000, 1977, 80125312789)
('p07', 'Cameron', 'Oscar', 44, 'Male', 'LO41AU', 'Single', 32000, 1978, 7888271209)
('p08', 'Claire', 'Alex', 40, 'Female', 'BO00MT', 'Married', 60000, 1974, 6409023476)
('p09', 'Marius', 'Tkozc', 29, 'Male', 'EX43AX', 'Single', 59000, 1993, 78945312789)
('p10', 'Justine', 'leo', 22, 'Male', 'Bl00AR', 'Single', 23000, 2000, 7561925836)
('p11', 'John', 'Cena', 48, 'Male', 'SW00BT', 'Married', 65000, 1974, 6403023742)
('p12', 'Rey', 'Mysterio', 60, 'Male', 'CA00TE', 'Married', 4

In [185]:
qry="""insert into ENCOUNTER (PATIENT_ID, ENCOUNTER_ID, DOCTOR_ID, ADMISSION_STATUS, BP_SYSTOLIC, BP_DIASTOLIC, PULSE, HEIGHT, WEIGHT, BMI)  
values 
("p01", "ENC0001", "DOC001", "Outpatient", "120", "80", "75", 180, 72, 20.8),
("p01", "ENC0002", "DOC001", "Inpatient", "130", "80", "75", 160, 65, 20.8),
("p02", "ENC0003", "DOC001", "Emergency", "140", "90", "95", 200, 155, 30.8),
("p02", "ENC0004", "DOC005", "Emergency", "140", "90", "50", 182, 73, 20.8),
("p03", "ENC0005", "DOC005", "Inpatient", "120", "80", "75", 170, 75, 20.6),
("p03", "ENC0006", "DOC005", "Inpatient", "120", "80", "75", 160, 75, 20.4),
("p03", "ENC0007", "DOC002", "Inpatient", "110", "80", "75", 182, 75, 20.1),
("p04", "ENC0008", "DOC001", "Inpatient", "110", "85", "75", 190, 77, 20.1),
("p05", "ENC0009", "DOC002", "Inpatient", "110", "80", "75", 200, 79, 20.3),
("p05", "ENC0010", "DOC001", "Inpatient", "120", "85", "75", 180, 139, 29.3),
("p06", "ENC0011", "DOC003", "Outpatient", "120", "85", "75", 155, 75, 24.8),
("p06", "ENC0012", "DOC001", "Outpatient", "120", "80", "75", 150, 87, 25.8),
("p07", "ENC0013", "DOC002", "Outpatient", "120", "80", "75", 170, 89, 25.7),
("p07", "ENC0014", "DOC003", "Inpatient", "120", "80", "75", 156, 60, 20.8),
("p07", "ENC0015", "DOC003", "Inpatient", "110", "80", "75", 178, 75, 20.8),
("p08", "ENC0016", "DOC004", "Inpatient", "120", "80", "75", 167, 63, 20.8),
("p09", "ENC0017", "DOC004", "Outpatient", "120", "80", "75", 180, 115, 27.8),
("p10", "ENC0018", "DOC004", "Outpatient", "120", "80", "75", 180, 75, 26.8),
("p10", "ENC0019", "DOC001", "Outpatient", "120", "80", "75", 180, 56, 20.8),
("p11", "ENC0020", "DOC001", "Inpatient", "120", "80", "75", 181, 67, 20.8),
("p12", "ENC0021", "DOC001", "Emergency", "140", "90", "60", 182, 59, 20.8),
("p13", "ENC0022", "DOC005", "Emergency", "130", "80", "65", 186, 97, 25.8),
("p13", "ENC0023", "DOC005", "Inpatient", "120", "80", "75", 166, 75, 20.8),
("p14", "ENC0024", "DOC005", "Inpatient", "110", "85", "75", 156, 89, 20.8),
("p15", "ENC0025", "DOC002", "Inpatient", "110", "85", "75", 187, 100, 24.8),
("p16", "ENC0026", "DOC002", "Inpatient", "110", "80", "75", 178, 75, 20.8),
("p17", "ENC0027", "DOC001", "Emergency", "125", "85", "65", 146, 172, 31),
("p18", "ENC0028", "DOC003", "Inpatient", "120", "80", "75", 180, 75, 20.8),
("p19", "ENC0029", "DOC004", "Outpatient", "120", "80", "75", 180, 75, 20.8),
("p19", "ENC0030", "DOC004", "Inpatient", "120", "80", "75", 180, 75, 20.8),
("p20", "ENC0031", "DOC004", "Inpatient", "120", "80", "75", 180, 95, 27.8),
("p20", "ENC0032", "DOC005", "Emergency", "140", "90", "70", 180, 98, 26.9),
("p21", "ENC0033", "DOC003", "Inpatient", "120", "80", "75", 170, 75, 24.8),
("p22", "ENC0034", "DOC003", "Inpatient", "110", "80", "75", 160, 78, 23.6),
("p23", "ENC0035", "DOC003", "Inpatient", "125", "80", "75", 150, 75, 20.4),
("p23", "ENC0036", "DOC004", "Inpatient", "125", "85", "75", 147, 72, 21.8),
("p24", "ENC0037", "DOC004", "Inpatient", "125", "80", "75", 186, 75, 20.9),
("p25", "ENC0038", "DOC004", "Inpatient", "120", "80", "75", 180, 65, 20.7),
("p26", "ENC0039", "DOC005", "Emergency", "120", "80", "70", 183, 59, 20.8),
("p27", "ENC0040", "DOC005", "Inpatient", "120", "80", "75", 180, 60, 20.8),
("p27", "ENC0041", "DOC006", "Outpatient", "120", "80", "75", 150, 75, 20.8),
("p27", "ENC0042", "DOC006", "Inpatient", "120", "80", "75", 200, 75, 28.8),
("p28", "ENC0043", "DOC004", "Outpatient", "120", "80", "75", 189, 75, 29.7),
("p28", "ENC0044", "DOC004", "Inpatient", "120", "80", "75", 188, 75, 20.8),
("p28", "ENC0045", "DOC002", "Emergency", "120", "85", "70", 187, 75, 20.8),
("p28", "ENC0046", "DOC002", "Inpatient", "120", "80", "75", 150, 68, 20.8),
("p29", "ENC0047", "DOC004", "Outpatient", "120", "80", "75", 140, 69, 20.8),
("p29", "ENC0048", "DOC004", "Inpatient", "120", "80", "75", 183, 75, 20.8),
("p30", "ENC0049", "DOC002", "Emergency", "140", "90", "65", 182, 98, 20.8),
("p30", "ENC0050", "DOC002", "Inpatient", "120", "80", "75", 161, 97, 24.8)
;"""

cur.execute(qry)

# Commit data to database
conn.commit()


# Prepare the query String
qry="select * from encounter;"

# Execute query on SQLite
cur.execute(qry)

# Display rows
rows=cur.fetchall()

for row in rows:
    print (row)

('ENC0001', 'p01', 'DOC001', 'Outpatient', 120, 80, 75, 180, 72, 20.8)
('ENC0002', 'p01', 'DOC001', 'Inpatient', 130, 80, 75, 160, 65, 20.8)
('ENC0003', 'p02', 'DOC001', 'Emergency', 140, 90, 95, 200, 155, 30.8)
('ENC0004', 'p02', 'DOC005', 'Emergency', 140, 90, 50, 182, 73, 20.8)
('ENC0005', 'p03', 'DOC005', 'Inpatient', 120, 80, 75, 170, 75, 20.6)
('ENC0006', 'p03', 'DOC005', 'Inpatient', 120, 80, 75, 160, 75, 20.4)
('ENC0007', 'p03', 'DOC002', 'Inpatient', 110, 80, 75, 182, 75, 20.1)
('ENC0008', 'p04', 'DOC001', 'Inpatient', 110, 85, 75, 190, 77, 20.1)
('ENC0009', 'p05', 'DOC002', 'Inpatient', 110, 80, 75, 200, 79, 20.3)
('ENC0010', 'p05', 'DOC001', 'Inpatient', 120, 85, 75, 180, 139, 29.3)
('ENC0011', 'p06', 'DOC003', 'Outpatient', 120, 85, 75, 155, 75, 24.8)
('ENC0012', 'p06', 'DOC001', 'Outpatient', 120, 80, 75, 150, 87, 25.8)
('ENC0013', 'p07', 'DOC002', 'Outpatient', 120, 80, 75, 170, 89, 25.7)
('ENC0014', 'p07', 'DOC003', 'Inpatient', 120, 80, 75, 156, 60, 20.8)
('ENC0015', 'p

In [188]:
qry="""insert into DIAGNOSIS (DIAGNOSIS_ID, PATIENT_ID, ENCOUNTER_ID, DIAGNOSIS_CODE, DIAGNOSIS_DESCRIPTION, DIAGNOSIS_DATE, DIAGNOSIS_CODE_TYPE)  
values 
("DG001","p01", "ENC0001", "I50.43", "Acute on chronic systolic (congestive) and diastolic (congestive) heart failure", "2021-05-24", "ICD10"),
("DG002","p02", "ENC0004", "I50.9", "Heart failure, Unspecified", "2020-05-03", "ICD10"),
("DG003","p03", "ENC0005", "I63", "Cerebral Infarction", "2020-12-20", "ICD10"),
("DG004","p04", "ENC0008", "I74.9", "Embolism and thrombosis of unspecified artery", "2020-04-05", "ICD10"),
("DG005","p05", "ENC0010", "E10.51", "Type 1 diabetes mellitus with diabetic peripheral angiopathy without gangrene", "2021-05-06", "ICD10"),
("DG006","p06", "ENC0011", "I70.243", "Atherosclerosis of native arteries of left leg with ulceration of ankle", "2021-11-12", "ICD10"),
("DG007","p07", "ENC0015", "I82.1", "Thrombophlebitis migrans", "2021-10-24", "ICD10"),
("DG008","p08", "ENC0016", "I86.4", "Gastric varices", "2020-05-23", "ICD10"),
("DG009","p09", "ENC0017", "S35.341", "Laceration of inferior mesenteric vein, sequela", "2020-06-21", "ICD10"),
("DG0010","p10", "ENC0018", "A39.83", "Meningococcal arthritis", "2022-01-02", "ICD10")
;"""

cur.execute(qry)

# Commit data to database
conn.commit()


# Prepare the query String
qry="select * from diagnosis;"

# Execute query on SQLite
cur.execute(qry)

# Display rows
rows=cur.fetchall()

for row in rows:
    print (row)

('DG001', 'p01', 'ENC0001', 'I50.43', 'Acute on chronic systolic (congestive) and diastolic (congestive) heart failure', '2021-05-24', 'ICD10')
('DG002', 'p02', 'ENC0004', 'I50.9', 'Heart failure, Unspecified', '2020-05-03', 'ICD10')
('DG003', 'p03', 'ENC0005', 'I63', 'Cerebral Infarction', '2020-12-20', 'ICD10')
('DG004', 'p04', 'ENC0008', 'I74.9', 'Embolism and thrombosis of unspecified artery', '2020-04-05', 'ICD10')
('DG005', 'p05', 'ENC0010', 'E10.51', 'Type 1 diabetes mellitus with diabetic peripheral angiopathy without gangrene', '2021-05-06', 'ICD10')
('DG006', 'p06', 'ENC0011', 'I70.243', 'Atherosclerosis of native arteries of left leg with ulceration of ankle', '2021-11-12', 'ICD10')
('DG007', 'p07', 'ENC0015', 'I82.1', 'Thrombophlebitis migrans', '2021-10-24', 'ICD10')
('DG008', 'p08', 'ENC0016', 'I86.4', 'Gastric varices', '2020-05-23', 'ICD10')
('DG009', 'p09', 'ENC0017', 'S35.341', 'Laceration of inferior mesenteric vein, sequela', '2020-06-21', 'ICD10')
('DG0010', 'p10',

In [190]:
qry="""insert into PROCEDURE (PROCEDURE_ID, PATIENT_ID, ENCOUNTER_ID, PROCEDURE_CODE, PROCEDURE_DESCRIPTION, PROCEDURE_START_DATE, PROCEDURE_END_DATE, PROCEDURE_CODE_TYPE)  
values 
("PR001","p01", "ENC0001", "Z48. 812", "Encounter for surgical aftercare following surgery on the circulatory system", "2020-07-24", "2020-07-24", "ICD10"),
("PR012","p02", "ENC0004", "4A023NZ", "Heart catheterization", "2020-05-05", "2020-05-05", "ICD10"),
("PR023","p03", "ENC0005", "F0134ZZ", "Motor Function Assessment of Neurological System - Whole Body", "2020-12-24", "2020-12-24", "ICD10"),
("PR034","p04", "ENC0008", "F020DYZ", "Neuromotor Development Assessment of Neurological System", "2020-04-02", "2020-04-02", "ICD10"),
("PR045","p05", "ENC0010", "D8000ZZ", "Beam Radiation of Eye using Photons ", "2021-05-07", "2021-05-08", "ICD10")
;"""

cur.execute(qry)

# Commit data to database
conn.commit()


# Prepare the query String
qry="select * from procedure;"

# Execute query on SQLite
cur.execute(qry)

# Display rows
rows=cur.fetchall()

for row in rows:
    print (row)

('PR001', 'p01', 'ENC0001', 'Z48.\xa0812', 'Encounter for surgical aftercare following surgery on the circulatory system', '2020-07-24', '2020-07-24', 'ICD10')
('PR012', 'p02', 'ENC0004', '4A023NZ', 'Heart catheterization', '2020-05-05', '2020-05-05', 'ICD10')
('PR023', 'p03', 'ENC0005', 'F0134ZZ', 'Motor Function Assessment of Neurological System - Whole Body', '2020-12-24', '2020-12-24', 'ICD10')
('PR034', 'p04', 'ENC0008', 'F020DYZ', 'Neuromotor Development Assessment of Neurological System', '2020-04-02', '2020-04-02', 'ICD10')
('PR045', 'p05', 'ENC0010', 'D8000ZZ', 'Beam Radiation of Eye using Photons\xa0', '2021-05-07', '2021-05-08', 'ICD10')


In [184]:
qry="""insert into HCP (DOCTOR_ID, DOCTOR_NAME, DEPARTMENT)
values 
("DOC001", "Franklin", "Cardiology"),
("DOC002", "Joe", "Neurology"),
("DOC003", "Alex", "Cardiology"),
("DOC004", "Hailey", "Gynecologists"),
("DOC005", "Olivia", "Family medicine"),
("DOC006", "Emma", "Orthopedic surgery")
;"""

cur.execute(qry)

# Commit data to database
conn.commit()


# Prepare the query String
qry="select * from hcp;"

# Execute query on SQLite
cur.execute(qry)

# Display rows
rows=cur.fetchall()

for row in rows:
    print (row)

('DOC001', 'Franklin', 'Cardiology')
('DOC002', 'Joe', 'Neurology')
('DOC003', 'Alex', 'Cardiology')
('DOC004', 'Hailey', 'Gynecologists')
('DOC005', 'Olivia', 'Family medicine')
('DOC006', 'Emma', 'Orthopedic surgery')


In [192]:
qry="""insert into BILLS (BILL_NO, PATIENT_ID, ENCOUNTER_ID, TOTAL_COST, NET_COST, INSURANCE_TYPE)  
values 
("B001","p01", "ENC0001", 50, 70, "NHS"),
("B002","p01", "ENC0002", 80, 100, "NHS"),
("B003","p02", "ENC0003", 30, 60, "Self Paid"),
("B004","p02", "ENC0004", 70, 110, "NHS"),
("B005","p03", "ENC0005", 40, 70, "Private"),
("B006","p03", "ENC0006", 20, 50, "NHS"),
("B007","p03", "ENC0007", 50, 80, "Self Paid"),
("B008","p04", "ENC0008", 100, 120, "NHS"),
("B009","p05", "ENC0009", 80, 100, "Private"),
("B010","p05", "ENC0010", 90, 120, "Private"),
("B011","p06", "ENC0011", 80, 100, "NHS"),
("B012","p06", "ENC0012", 40, 70, "Self Paid"),
("B013","p07", "ENC0013", 30, 60, "NHS"),
("B014","p07", "ENC0014", 20, 40, "NHS"),
("B015","p07", "ENC0015", 50, 70, "Private"),
("B016","p08", "ENC0016", 60, 90, "NHS"),
("B017","p09", "ENC0017", 70, 90, "Private"),
("B018","p10", "ENC0018", 80, 100, "NHS"),
("B019","p10", "ENC0019", 100, 120, "Private"),
("B020","p11", "ENC0020", 30, 40, "NHS"),
("B021","p12", "ENC0021", 40, 60, "NHS"),
("B022","p13", "ENC0022", 50, 80, "Self Paid"),
("B023","p13", "ENC0023", 20, 40, "NHS"),
("B024","p14", "ENC0024", 70, 90, "NHS"),
("B025","p15", "ENC0025", 80, 100, "Private"),
("B026","p16", "ENC0026", 100, 110, "Private"),
("B027","p17", "ENC0027", 100, 120, "Private"),
("B028","p18", "ENC0028", 20, 30, "NHS"),
("B029","p19", "ENC0029", 50, 70, "NHS"),
("B030","p19", "ENC0030", 60, 80, "NHS"),
("B031","p20", "ENC0031", 90, 110, "NHS"),
("B032","p20", "ENC0032", 30, 50, "Self Paid"),
("B033","p21", "ENC0033", 80, 100, "Self Paid"),
("B034","p22", "ENC0034", 40, 70, "NHS"),
("B035","p23", "ENC0035", 70, 90, "NHS"),
("B036","p23", "ENC0036", 110, 130, "Self Paid"),
("B037","p24", "ENC0037", 30, 50, "NHS"),
("B038","p25", "ENC0038", 40, 60, "NHS"),
("B039","p26", "ENC0039", 70, 90, "Private"),
("B040","p27", "ENC0040", 20, 50, "NHS"),
("B041","p27", "ENC0041", 60, 80, "NHS"),
("B042","p27", "ENC0042", 30, 40, "Self Paid"),
("B043","p28", "ENC0042", 100, 110, "NHS"),
("B044","p28", "ENC0044", 80, 90, "NHS"),
("B045","p28", "ENC0045", 50, 70, "Private"),
("B046","p28", "ENC0046", 30, 50, "NHS"),
("B047","p29", "ENC0047", 80, 90, "NHS"),
("B048","p29", "ENC0048", 90, 110, "Self Paid"),
("B049","p30", "ENC0049", 30, 50, "Self Paid"),
("B050","p30", "ENC0050", 50, 70, "NHS")
;"""

cur.execute(qry)

# Commit data to database
conn.commit()


# Prepare the query String
qry="select * from bills;"

# Execute query on SQLite
cur.execute(qry)

# Display rows
rows=cur.fetchall()

for row in rows:
    print (row)

('B001', 'p01', 'ENC0001', 50, 70, 'NHS')
('B002', 'p01', 'ENC0002', 80, 100, 'NHS')
('B003', 'p02', 'ENC0003', 30, 60, 'Self Paid')
('B004', 'p02', 'ENC0004', 70, 110, 'NHS')
('B005', 'p03', 'ENC0005', 40, 70, 'Private')
('B006', 'p03', 'ENC0006', 20, 50, 'NHS')
('B007', 'p03', 'ENC0007', 50, 80, 'Self Paid')
('B008', 'p04', 'ENC0008', 100, 120, 'NHS')
('B009', 'p05', 'ENC0009', 80, 100, 'Private')
('B010', 'p05', 'ENC0010', 90, 120, 'Private')
('B011', 'p06', 'ENC0011', 80, 100, 'NHS')
('B012', 'p06', 'ENC0012', 40, 70, 'Self Paid')
('B013', 'p07', 'ENC0013', 30, 60, 'NHS')
('B014', 'p07', 'ENC0014', 20, 40, 'NHS')
('B015', 'p07', 'ENC0015', 50, 70, 'Private')
('B016', 'p08', 'ENC0016', 60, 90, 'NHS')
('B017', 'p09', 'ENC0017', 70, 90, 'Private')
('B018', 'p10', 'ENC0018', 80, 100, 'NHS')
('B019', 'p10', 'ENC0019', 100, 120, 'Private')
('B020', 'p11', 'ENC0020', 30, 40, 'NHS')
('B021', 'p12', 'ENC0021', 40, 60, 'NHS')
('B022', 'p13', 'ENC0022', 50, 80, 'Self Paid')
('B023', 'p13', 'E

In [194]:
qry="""insert into pharmacy (PURCHASE_ID, PATIENT_ID, ENCOUNTER_ID, DRUG_CODE, DRUG_NAME, DOSAGE, FREQUENCY)
values 
("PUR101", "p01", "ENC0001", "NDC077-3105-01", "Kivexa", "100", 3),
("PUR102", "p01", "ENC0002", "NDC077-3105-02", "Trizivir", "200", 2),
("PUR103", "p02", "ENC0003", "NDC077-3105-03", "Abarelix", "250", 1),
("PUR104", "p02", "ENC0004", "NDC077-3105-04", "Actictin", "300", 1),
("PUR105", "p03", "ENC0005", "NDC077-3105-05", "Asacol", "320", 2),
("PUR106", "p03", "ENC0006", "NDC077-3105-06", "Ascleara", "450", 3),
("PUR107", "p03", "ENC0007", "NDC077-3105-07", "Exemestane", "420", 2),
("PUR108", "p04", "ENC0008", "NDC077-3105-08", "Nuvigil", "600", 2),
("PUR109", "p05", "ENC0009", "NDC077-3105-09", "Forte", "120", 3),
("PUR110", "p05", "ENC0010", "NDC077-3105-10", "Aristocort", "240", 3),
("PUR111", "p06", "ENC0011", "NDC077-3105-11", "Iscepto-p", "100", 3),
("PUR112", "p06", "ENC0012", "NDC077-3105-12", "Abilify", "200", 2),
("PUR113", "p07", "ENC0013", "NDC077-3105-13", "Kivexa", "250", 1),
("PUR121", "p07", "ENC0014", "NDC077-3105-14", "Trizivir", "300",1),
("PUR122", "p07", "ENC0015", "NDC077-3105-15", "Abarelix", "320", 2),
("PUR123", "p08", "ENC0016", "NDC077-3105-16", "Actictin", "450", 3),
("PUR124", "p09", "ENC0017", "NDC077-3105-17", "Asacol", "100", 2),
("PUR125", "p10", "ENC0018", "NDC077-3105-18", "Ascleara", "200", 2),
("PUR126", "p10", "ENC0019", "NDC077-3105-19", "Kivexa", "250", 3),
("PUR127", "p11", "ENC0020", "NDC077-3105-20", "Trizivir", "300", 3),
("PUR128", "p12", "ENC0021", "NDC077-3105-21", "Abarelix", "320", 3),
("PUR129", "p13", "ENC0022", "NDC077-3105-22", "Actictin", "450", 2),
("PUR130", "p13", "ENC0023", "NDC077-3105-23", "Asacol", "420", 1),
("PUR131", "p14", "ENC0024", "NDC077-3105-24", "Kivexa", "600", 1),
("PUR132", "p15", "ENC0025", "NDC077-3105-25", "Trizivir", "120", 2),
("PUR133", "p16", "ENC0026", "NDC077-3105-26", "Abarelix", "240", 3),
("PUR134", "p17", "ENC0027", "NDC077-3105-27", "Actictin", "100", 2),
("PUR135", "p18", "ENC0028", "NDC077-3105-28", "Asacol", "200", 2),
("PUR136", "p19", "ENC0029", "NDC077-3105-29", "Kivexa", "250", 3),
("PUR137", "p19", "ENC0030", "NDC077-3105-30", "Trizivir", "300", 3),
("PUR138", "p20", "ENC0031", "NDC077-3105-31", "Abarelix", "320", 3),
("PUR139", "p20", "ENC0032", "NDC077-3105-32", "Actictin", "450", 2),
("PUR140", "p21", "ENC0033", "NDC077-3105-33", "Asacol", "420", 1),
("PUR141", "p22", "ENC0034", "NDC077-3105-34", "Ascleara", "100", 1),
("PUR151", "p23", "ENC0035", "NDC077-3105-35", "Exemestane", "200", 2),
("PUR161", "p23", "ENC0036", "NDC077-3105-36", "Nuvigil", "100", 3),
("PUR171", "p24", "ENC0037", "NDC077-3105-37", "Forte", "200", 2),
("PUR181", "p25", "ENC0038", "NDC077-3105-38", "Aristocort", "250", 2),
("PUR191", "p26", "ENC0039", "NDC077-3105-39", "Iscepto-p", "300", 3),
("PUR192", "p27", "ENC0040", "NDC077-3105-40", "Abilify", "320", 3),
("PUR193", "p27", "ENC0041", "NDC077-3105-41", "Kivexa", "450",    3),
("PUR194", "p27", "ENC0042", "NDC077-3105-42", "Trizivir", "420", 2),
("PUR195", "p28", "ENC0043", "NDC077-3105-43", "Abarelix", "600", 1),
("PUR196", "p28", "ENC0044", "NDC077-3105-44", "Actictin", "100", 1),
("PUR197", "p28", "ENC0045", "NDC077-3105-45", "Asacol", "200", 2),
("PUR198", "p28", "ENC0046", "NDC077-3105-46", "Ascleara", "250", 3),
("PUR199", "p29", "ENC0047", "NDC077-3105-47", "Exemestane", "300", 2),
("PUR172", "p29", "ENC0048", "NDC077-3105-48", "Nuvigil", "320", 2),
("PUR173", "p30", "ENC0049", "NDC077-3105-49", "Forte", "450", 3),
("PUR174", "p30", "ENC0050", "NDC077-3105-50", "Aristocort", "420", 3)
;"""

cur.execute(qry)

# Commit data to database
conn.commit()


# Prepare the query String
qry="select * from pharmacy;"

# Execute query on SQLite
cur.execute(qry)

# Display rows
rows=cur.fetchall()

for row in rows:
    print (row)

('PUR101', 'p01', 'ENC0001', 'NDC077-3105-01', 'Kivexa', '100', '3')
('PUR102', 'p01', 'ENC0002', 'NDC077-3105-02', 'Trizivir', '200', '2')
('PUR103', 'p02', 'ENC0003', 'NDC077-3105-03', 'Abarelix', '250', '1')
('PUR104', 'p02', 'ENC0004', 'NDC077-3105-04', 'Actictin', '300', '1')
('PUR105', 'p03', 'ENC0005', 'NDC077-3105-05', 'Asacol', '320', '2')
('PUR106', 'p03', 'ENC0006', 'NDC077-3105-06', 'Ascleara', '450', '3')
('PUR107', 'p03', 'ENC0007', 'NDC077-3105-07', 'Exemestane', '420', '2')
('PUR108', 'p04', 'ENC0008', 'NDC077-3105-08', 'Nuvigil', '600', '2')
('PUR109', 'p05', 'ENC0009', 'NDC077-3105-09', 'Forte', '120', '3')
('PUR110', 'p05', 'ENC0010', 'NDC077-3105-10', 'Aristocort', '240', '3')
('PUR111', 'p06', 'ENC0011', 'NDC077-3105-11', 'Iscepto-p', '100', '3')
('PUR112', 'p06', 'ENC0012', 'NDC077-3105-12', 'Abilify', '200', '2')
('PUR113', 'p07', 'ENC0013', 'NDC077-3105-13', 'Kivexa', '250', '1')
('PUR121', 'p07', 'ENC0014', 'NDC077-3105-14', 'Trizivir', '300', '1')
('PUR122', '

### Displaying tables

In [195]:
def tables_in_sqlite_db(conn):
    cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [
        v[0] for v in cursor.fetchall()
        if v[0] != "sqlite_sequence"
    ]
    cursor.close()
    return tables,

In [196]:
# call the function and pass connection object 
tables = tables_in_sqlite_db(conn)

#print tables in the current database
print(tables)

(['PATIENT', 'ENCOUNTER', 'HCP', 'DIAGNOSIS', 'PROCEDURE', 'BILLS', 'PHARMACY'],)


### Updating marital status of a patient

In [255]:
inputptid=input('Enter the patient id that you wish to change:')
inputmarital=input('Enter Marital status (Single, Married, Widow):')
qry='update patient set MARITAL_STATUS=? where PATIENT_ID=?'

try:
        cur.execute(qry, (inputmarital,inputptid))
        print ('Marital Status is updated.')
        conn.commit()
except:
        print ('Error in update operation... rollback')
        conn.rollback()

Enter the patient id that you wish to change:p27
Enter Marital status (Single, Married, Widow):Married
Marital Status is updated.


### Verifying the update for Marital Status

In [256]:
qry="select * from patient where PATIENT_ID = 'p27';"

# Execute query on SQLite
cur.execute(qry)

# Display rows
rows=cur.fetchall()

for row in rows:
    print (row)

('p27', 'Boriss', 'Thomsan', 24, 'Male', 'LN41AU', 'Married', 12000, 1998, 7778881253)


### Deleting records from pharmacy table when the purchase has been cancelled by the patient

In [207]:
inputName=input('Enter the purchase id:')
qry='delete from pharmacy where PURCHASE_ID=?'
try:
        cur.execute(qry, (inputName,))
        print ('Order successfully deleted')
        conn.commit()
except:
        print ('Error in deleting order', inputName)
        conn.rollback()


Enter the purchase id:PUR174
Order successfully deleted


### Displaying the total NETCOST  the patient has spent in a new column

In [209]:
qry="""Select BILL_NO, PATIENT_ID, ENCOUNTER_ID, TOTAL_COST, sum(NET_COST) as patient_net_cost, INSURANCE_TYPE
from bills
GROUP BY PATIENT_ID;"""
cur.execute(qry)


# Commit data to database
conn.commit()

# Display rows
rows=cur.fetchall()

for row in rows:
    print (row)

('B001', 'p01', 'ENC0001', 50, 170, 'NHS')
('B003', 'p02', 'ENC0003', 30, 170, 'Self Paid')
('B005', 'p03', 'ENC0005', 40, 200, 'Private')
('B008', 'p04', 'ENC0008', 100, 120, 'NHS')
('B009', 'p05', 'ENC0009', 80, 220, 'Private')
('B011', 'p06', 'ENC0011', 80, 170, 'NHS')
('B013', 'p07', 'ENC0013', 30, 170, 'NHS')
('B016', 'p08', 'ENC0016', 60, 90, 'NHS')
('B017', 'p09', 'ENC0017', 70, 90, 'Private')
('B018', 'p10', 'ENC0018', 80, 220, 'NHS')
('B020', 'p11', 'ENC0020', 30, 40, 'NHS')
('B021', 'p12', 'ENC0021', 40, 60, 'NHS')
('B022', 'p13', 'ENC0022', 50, 120, 'Self Paid')
('B024', 'p14', 'ENC0024', 70, 90, 'NHS')
('B025', 'p15', 'ENC0025', 80, 100, 'Private')
('B026', 'p16', 'ENC0026', 100, 110, 'Private')
('B027', 'p17', 'ENC0027', 100, 120, 'Private')
('B028', 'p18', 'ENC0028', 20, 30, 'NHS')
('B029', 'p19', 'ENC0029', 50, 150, 'NHS')
('B031', 'p20', 'ENC0031', 90, 160, 'NHS')
('B033', 'p21', 'ENC0033', 80, 100, 'Self Paid')
('B034', 'p22', 'ENC0034', 40, 70, 'NHS')
('B035', 'p23', 

### Joining diagnosis table with encounter table to understand the Patient's vital condition

In [210]:
qry="""SELECT a.*, b.pulse, b.height, b.weight, b.bmi, b.bp_systolic, b.bp_diastolic
FROM diagnosis a
inner JOIN encounter b
ON a.encounter_id = b.encounter_id;"""
cur.execute(qry)


# Commit data to database
conn.commit()

# Display rows
rows=cur.fetchall()

for row in rows:
    print (row)

('DG001', 'p01', 'ENC0001', 'I50.43', 'Acute on chronic systolic (congestive) and diastolic (congestive) heart failure', '2021-05-24', 'ICD10', 75, 180, 72, 20.8, 120, 80)
('DG002', 'p02', 'ENC0004', 'I50.9', 'Heart failure, Unspecified', '2020-05-03', 'ICD10', 50, 182, 73, 20.8, 140, 90)
('DG003', 'p03', 'ENC0005', 'I63', 'Cerebral Infarction', '2020-12-20', 'ICD10', 75, 170, 75, 20.6, 120, 80)
('DG004', 'p04', 'ENC0008', 'I74.9', 'Embolism and thrombosis of unspecified artery', '2020-04-05', 'ICD10', 75, 190, 77, 20.1, 110, 85)
('DG005', 'p05', 'ENC0010', 'E10.51', 'Type 1 diabetes mellitus with diabetic peripheral angiopathy without gangrene', '2021-05-06', 'ICD10', 75, 180, 139, 29.3, 120, 85)
('DG006', 'p06', 'ENC0011', 'I70.243', 'Atherosclerosis of native arteries of left leg with ulceration of ankle', '2021-11-12', 'ICD10', 75, 155, 75, 24.8, 120, 85)
('DG007', 'p07', 'ENC0015', 'I82.1', 'Thrombophlebitis migrans', '2021-10-24', 'ICD10', 75, 178, 75, 20.8, 110, 80)
('DG008', 'p

### Installing MongoDB Engine

In [212]:
import pymongo
from pymongo import MongoClient

In [213]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [214]:
mg=MongoClient('localhost', 27017)

mongo_db = mg.ehr_db_mongo #create new db / connect to existing

In [229]:
#create new collections

reports = mongo_db.reports


In [246]:
reports.insert_one({'_id':'p01','issue':'chest infection','medical history':'no past issues',
                            'report': 'surgery is necessary for mild cardiac failure. Lower blood pressure. When feet are examined, they show no signs of damaged skin or calluses. There have been no reported visual issues. High glucometer readings, suboptimal sugar, and control of retinopathy and neuropathy. should put more effort into my nutrition. will boost insulin by 2 units, be audible and percussion-free, and have normal respiratory effort',
                           'treatment': 'Physiotherapy and Angioplasty', 'medicine': 'Angiotensin-2 receptor blocker',
                           'frequency': '3 times a day'})
reports.insert_one({'_id':'p02','issue':'Fractured Ankle','medical history':'no past issues',
                            'report': 'If you have a displaced fracture, which means the two ends of the fracture are not perfectly aligned, your doctor may need to move the fragments back into place. This is known as reduction. Depending on the severity of your pain and swelling, you may require a muscle relaxant, sedative, or local anaesthetic to numb the region before this treatment.',
                           'treatment': 'Ankle  orthopedic Surgery', 'medicine': 'Angiotensin-converting enzyme (ACE) inhibitors',
                           'frequency': '2 times a day'})
reports.insert_one({'_id':'p03','issue':'Lump inside lungs','medical history':'no past issues',
                            'report': 'Malignant tumours occur as a result of uncontrolled multiplication and proliferation of lung cells. The illness is distinguished by a persistent cough, the presence of blood in the cough, and shortness of breath. Surgery, chemotherapy, and radiation therapy are used in the treatment.','treatment': 'Surgery - chemotherapy', 'medicine': 'Angiotensin-2 receptor blocker',
                           'frequency': '3 times a day'})
reports.insert_one({'_id':'p04','issue':'Tumour Inside Brain','medical history':'no past issues',
                            'report': 'A swarm of aberrant brain cells. This causes headaches, nausea, visual issues, progressive loss of feeling, hearing impairments, and seizures. Oncogenic tumours grow from the membranes that surround the brain.','treatment': 'Open Brain Surgery, chemotherapy, Radiation therapy', 'medicine': 'Synaptid, Codeage',
                           'frequency': '3 times a day'})
reports.insert_one({'_id':'p05','issue':'Runny Nose and Cold','medical history':'no past issues',
                            'report': 'Viral infection will be to cause. Rhinovirus is the most common cause. The most frequent symptoms are nasal stuffiness, runny nose, sore throat, sneezing, and fever.','treatment': 'Steaming', 'medicine': 'Aspirin','frequency': '2 times a day'})
reports.insert_one({'_id':'p06','issue':'Seizure','medical history':'no past issues',
                            'report': 'An uncontrollable electrical disruption in the brain that can result in changes in behaviour, movements, sensations, and awareness.','treatment': 'Steaming', 'medicine': 'Levetiracetam','frequency': '2 times a day'})
reports.insert_one({'_id':'p08','issue':'Liver Gallstones','medical history':'no past issues',
                            'report': 'The gallbladder produces tone from precipitated bile components. Cholelithiasis refers to the presence of gallstones or any condition induced by gallstones, whereas choledocholithiasis refers to the presence of migrating gallstones inside the bile ducts.','treatment':'Liver Surgery', 'medicine': 'Isoniazid','frequency': '3 times a day'})
reports.insert_one({'_id':'p09','issue':'Liver Failure','medical history':'no past issues',
                            'report': 'Rapid loss of liver function that happens in days or weeks in people who do not have underlying liver disease. The occurrence of serious consequences soon after the initial indications of liver illness (such as jaundice), indicating that the liver has suffered substantial damage (loss of function of 80-90 percent of liver cells).','treatment': 'Liver Surgery', 'medicine': 'Isoniazid','frequency': '3 times a day'})
reports.insert_one({'_id':'p11','issue':'Haemochromatosis','medical history':'no past issues',
                            'report': 'Excess iron in the body causes dangerous illnesses such diabetes, heart disease, and liver damage. Haemochromatosis results from a genetic mutation. Common symptoms include joint discomfort, tiredness, and liver failure. It is effectively treated by extracting blood to lower the iron level.','treatment': 'Chelation therapy', 'medicine': 'Deferoxamine','frequency': '2 times a day'})
reports.insert_one({'_id':'p13','issue':'Runny Nose and Cold','medical history':'no past issues',
                            'report': 'Viral infection will be to cause. Rhinovirus is the most common cause. The most frequent symptoms are nasal stuffiness, runny nose, sore throat, sneezing, and fever. Treatment include controlling indications and symptoms.','treatment': 'Nasal sprays', 'medicine': 'Otrivin Spray','frequency': '3 times a day'})
reports.insert_one({'_id':'p14','issue':'Stroke','medical history':'no past issues','report': 'The symptoms, medical history, blood pressure, and blood vessels at the back of the eyes of the patient are all evaluated. Dyes are administered to provide a more detailed image of the brain and neck blood arteries seen on X-ray.','treatment': 'therapy', 'medicine': 'Clopidogrel','frequency': '2 times a day'})
reports.insert_one({'_id':'p15','issue':'aortic aneurysms','medical history':'no past issues','report': 'Rheumatic fever is a dangerous symptom because it can develop to persistent RHD with its attendant consequences of atrial fibrillation, stroke, heart failure, and infective endocarditis.','treatment': 'Cardiac surgery', 'medicine': 'Plasminogen activator (tPA)','frequency': '2 times a day'})
reports.insert_one({'_id':'p16','issue':'peripheral artery disease','medical history':'no past issues','report': 'A minor hereditary condition in which the liver fails to appropriately handle bilirubin. The breakdown of red blood cells produces bilirubin.','treatment': 'Cardiac surgery', 'medicine': 'Plasminogen activator (tPA)','frequency': '2 times a day'})
reports.insert_one({'_id':'p17','issue':'thromboembolic disease','medical history':'no past issues','report': 'A minor hereditary condition in which the liver fails to appropriately handle bilirubin. The breakdown of red blood cells produces bilirubin.','treatment': 'Cardiac surgery', 'medicine': 'Plasminogen activator (tPA)','frequency': '2 times a day'})
reports.insert_one({'_id':'p18','issue':'vericos vein','medical history':'no past issues','report': 'Veins feature bivalved valves that guide blood flow to the heart. A venous valve dysfunction causes blood to pool and obstructs flow. This is characterised by clearly enlarged and twisted veins. It is normally asymptomatic, although you may detect discoloration or enlarged veins, particularly in your legs and feet.','treatment': 'Venal Incubation', 'medicine': 'Venal activator (tPA)','frequency': '3 times a day'})
reports.insert_one({'_id':'p19','issue':'Chronic kidney disease','medical history':'The kidneys are working at less than 15% of their typical capacity. Renal failure is divided into two types: acute kidney failure, which occurs quickly and may resolve, and chronic kidney failure.','treatment': 'Kidney Transplant', 'medicine': 'Paracetamol','frequency': '2 times a day'})
reports.insert_one({'_id':'p20','issue':'Chronic kidney disease','medical history':'The kidneys are working at less than 15% of their typical capacity. Renal failure is divided into two types: acute kidney failure, which occurs quickly and may resolve, and chronic kidney failure.','treatment': 'Kidney Transplant', 'medicine': 'Paracetamol','frequency': '2 times a day'})
reports.insert_one({'_id':'p21','issue':'Fractured Ankle','medical history':'If you have a displaced fracture, which means the two ends of the fracture are not perfectly aligned, your doctor may need to move the fragments back into place. This is known as reduction. Depending on the severity of your pain and swelling, you may require a muscle relaxant, sedative, or local anaesthetic to numb the region before this treatment.','treatment': 'Ankle  orthopedic Surgery', 'medicine': 'Angiotensin-2 receptor blocker','frequency': '2 times a day'})
reports.insert_one({'_id':'p22','issue':'Dislocated Elbow','medical history':'If you have a displaced fracture, which means the two ends of the fracture are not perfectly aligned, your doctor may need to move the fragments back into place. This is known as reduction. Depending on the severity of your pain and swelling, you may require a muscle relaxant, sedative, or local anaesthetic to numb the region before this treatment.','treatment': 'Surgery', 'medicine': 'Angiotensin-2 receptor blocker','frequency': '2 times a day'})
reports.insert_one({'_id':'p23','issue':'Dislocated Jaw','medical history':'If you have a displaced fracture, which means the two ends of the fracture are not perfectly aligned, your doctor may need to move the fragments back into place. This is known as reduction. Depending on the severity of your pain and swelling, you may require a muscle relaxant, sedative, or local anaesthetic to numb the region before this treatment.','treatment': 'Surgery', 'medicine': 'Angiotensin-2 receptor blocker','frequency': '2 times a day'})
reports.insert_one({'_id':'p24','issue':'Thrombosis','medical history':'The presence or creation of a blood clot in a blood vessel. A vessel can be any vein or artery, as in deep vein thrombosis or coronary (artery) thrombosis. The clot is known as a thrombus.','treatment': 'Surgery', 'medicine': 'Angiotensin-2 receptor blocker','frequency': '2 times a day'})
reports.insert_one({'_id':'p25','issue':'Ventricular Thrombosis','medical history':'The presence or creation of a blood clot in a blood vessel. A vessel can be any vein or artery, as in deep vein thrombosis or coronary (artery) thrombosis. The clot is known as a thrombus.','treatment': 'Surgery', 'medicine': 'Angiotensin-2 receptor blocker','frequency': '2 times a day'})
reports.insert_one({'_id':'p26','issue':'Tuberculosis','medical history':'Lymph nodes for edoema and lungs noises A skin test is usually performed to confirm the presence of germs. Blood and sputum testing are additional options.','treatment': 'Lymph surgery', 'medicine': 'Angiotensin-2 receptor blocker','frequency': '3 times a day'})
reports.insert_one({'_id':'p27','issue':'Muscular dystrophy','medical history':'Changes in the genes that govern muscle stability and function Progressive muscle weakening is one of the most prevalent signs of muscular dystrophy, along with difficulties walking, muscle discomfort, and stiffness.','treatment': 'Muscular scopy', 'medicine': 'Angiotensin-2 receptor blocker','frequency': '3 times a day'})
reports.insert_one({'_id':'p28','issue':'Meningitis','medical history':'Loss of appetite, fever, tiredness, lethargy, stiff neck, and headache are all symptoms. Mild infections heal in 2 weeks. In extreme situations, medication and therapy are required. Bacterial or fungal meningitis is usually fatal and necessitates prompt diagnosis and treatment.','treatment': 'Radiation Therapy', 'medicine': 'Angiotensin-2 receptor blocker','frequency': '3 times a day'})
reports.insert_one({'_id':'p29','issue':'Melanoma','medical history':'no past issues','report':'Cancer of the skin (melanoma) Melanoma is a kind of skin cancer that has the potential to spread to other organs. The emergence of a new mole or a change in an existing mole is the most typical symptom of melanoma. This may happen everywhere on the body, but the back and legs are the most typically afflicted areas in men.','treatment': 'Radiation Therapy', 'medicine': 'Angiotensin-2 receptor blocker','frequency': '3 times a day'})
reports.insert_one({'_id':'p30','issue':'Glaucoma','medical history':'no past issues','report':'With or without increased intraocular pressure, the optic nerve of the eye, which transmits information to the brain, is destroyed. If left untreated, this will result in progressive visual loss.','treatment': 'Laser Therapy', 'medicine': 'Angiotensin-2 receptor blocker','frequency': '3 times a day'})



### Updating the document

In [257]:
newvalues = { "$set": { "issue": 'Cancer','medical history': 'cancer in the past'} }
reports.update_one({"_id":'p30'}, newvalues)